#### The first thing we'll do is installing required packages. Press Shift + Enter to make the cell run

In [ ]:
pip install panda

In [ ]:
pip install yahoo-fantasy-api

In [5]:
pip install nested-lookup

Note: you may need to restart the kernel to use updated packages.


####  After pressing Shift+Enter the below cell. Click and open the authorization URL.
#### Accept and copy the code to the "Enter verifier" field. The error message will show "KeyError: 'access_token" if the code is not correct, Shift-Enter this cell again.

In [37]:
import yahoo_fantasy_api as yfa
import pandas as pd
from yahoo_oauth import OAuth2
import json

creds = {'consumer_key': 'dj0yJmk9dXpGMDNlZjhMYWJqJmQ9WVdrOU4ycE9PV1pUUXpnbWNHbzlNQT09JnM9Y29uc3VtZXJzZWNyZXQmc3Y9MCZ4PTEw',
         'consumer_secret': 'e82a0d6df8f79c59a94243b908dcbb16ed4dc0da'}
with open('oauth2.json', "w") as f:
    f.write(json.dumps(creds))
oauth = OAuth2(None, None, from_file='oauth2.json')

[2024-03-19 02:34:33,766 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2024-03-19 02:34:33,769 DEBUG] [yahoo_oauth.oauth.handler] AUTHORIZATION URL : https://api.login.yahoo.com/oauth2/request_auth?redirect_uri=oob&response_type=code&client_id=dj0yJmk9dXpGMDNlZjhMYWJqJmQ9WVdrOU4ycE9PV1pUUXpnbWNHbzlNQT09JnM9Y29uc3VtZXJzZWNyZXQmc3Y9MCZ4PTEw


Enter verifier : vxt5t95


#### Find the league_id

In [38]:
print(type(oauth))
print(oauth.__dict__)

oauth1 = oauth
gm = yfa.game.Game(oauth, 'nba')
gm.league_ids(year=2023)

<class 'yahoo_oauth.oauth.OAuth2'>
{'oauth_version': 'oauth2', 'from_file': 'oauth2.json', 'consumer_key': 'dj0yJmk9dXpGMDNlZjhMYWJqJmQ9WVdrOU4ycE9PV1pUUXpnbWNHbzlNQT09JnM9Y29uc3VtZXJzZWNyZXQmc3Y9MCZ4PTEw', 'consumer_secret': 'e82a0d6df8f79c59a94243b908dcbb16ed4dc0da', 'callback_uri': 'oob', 'store_file': True, 'browser_callback': True, 'oauth': <rauth.service.OAuth2Service object at 0x1225d4b20>, 'verifier': 'vxt5t95', 'token_time': 1710783283.2678359, 'access_token': 'oPDLZjSY41E.U.nDaUqy1nq0.wMqqyusO7ZwJGJ.yReDEhh2ZmI.gvqdm4fdb_03KRpMBHg3ZbDB.F83lZfHkJl5nkMBZsRvGQTlVhq4fuTG4.eI5N4OjM0e0Ne9QMa58_gdHJAhhpgPM4srpgdVf7Ey1MZamDj6JTH5yDd4TLVcAY9rHSC0wOxIV_Qiz0vhPwdVXEEPK61B6dOIUTXf6svzeb9Vl0Kj0cUMnKoLAQ2ywMKWS.LBEaZ8aLqpJL1Q2QlPaADhs5DQKcUy.QPQ2TjmxPNumRC_nJ1gwv2XD9.YA9fhdAOB.C641WdFrnlSZN3ilGtCM1kldW94nvhIOcfxzgv5voJwGWpmmV7YgqvQFlqFrkPgp9TGb1khsDkT2AwRct7QSkZUImbvAx9TJovPb2kltKFs.aVXRl32ep_flyTQSlr5eITxuXSsLmj8w6ltHCLt6ur5BDyX0tScSBQQw.35Q5UZ2ST8s5JRuqfmPiJyZlcrqjrh785OgLBJoMjlEQK2lwhaR

['428.l.44884']

#### Replace your league_id to parameter field. An example is '402.l.28035'

In [39]:
##- C1: 6730
##- SC: 35992
##- Dynasty: 4273
##- PL: 36389
##- SA: 35254
##- CSL: 35264
##- Bundes: 35898
##- VL: 43386
##- L2: 43399
"""
List of league 

22103	NBA Champions League 21st 2024
76043	Super Cup 22-23
40982	NBA Talk VN - Premier League
40984	NBA Talk VN - Serie A
44737	NBA Talk VN - Bundesliga
44884	NBA Talk VN - RSL
108219	NBA Talk VN - VLeague
29668	NBA Talk VN - 29668
"""
lg = gm.to_league('428.l.76043')

#### Before running it, change the week you want to ccess and obtain league stats.

In [41]:
import dataframe_image as dfi
from nested_lookup import nested_lookup
matchup = lg.matchups(week=20)
week_plot = 20
stat_name = ["Team","FG%","FT%","3PTM","PTS","REB","AST","ST","BLK","TO","Score"]
league_stat = pd.DataFrame(columns=stat_name)


#Obtain the league stats
name = nested_lookup('name', matchup)
team_stats = nested_lookup('team_stats', matchup)
team_points = nested_lookup('team_points', matchup)


no_team = len(lg.teams().keys())
#print(matchup)
for i in range (8):
    stat = nested_lookup('value', team_stats[i])
    point = nested_lookup('total', team_points[i])
    
    stat = list(map(str,stat))
    stat = ['0' if x=='-' else x for x in stat]
    stat = list(map(float,[ x for x in stat if '/' not in x ]))
    point = list(map(float,point))
    full_stat = [name[i+1]] + stat + point
    league_stat.loc[len(league_stat)] = full_stat
    
#Calculate the roto ranks
league_stat['FG%_rank']=league_stat['FG%'].rank(ascending=True, method='min')
league_stat['FT%_rank']=league_stat['FT%'].rank(ascending=True, method='min')    
league_stat['3PTM_rank']=league_stat['3PTM'].rank(ascending=True, method='min')    
league_stat['PTS_rank']=league_stat['PTS'].rank(ascending=True, method='min')    
league_stat['REB_rank']=league_stat['REB'].rank(ascending=True, method='min')    
league_stat['AST_rank']=league_stat['AST'].rank(ascending=True, method='min')    
league_stat['ST_rank']=league_stat['ST'].rank(ascending=True, method='min')    
league_stat['BLK_rank']=league_stat["BLK"].rank(ascending=True, method='min')      
league_stat['TO_rank']=league_stat['TO'].rank(ascending=False, method='min')    
league_stat['Total']= league_stat['FG%_rank']+league_stat['FT%_rank']+league_stat['3PTM_rank']+league_stat['REB_rank']+league_stat['PTS_rank'] + \
league_stat['AST_rank']  + league_stat['ST_rank'] + league_stat['BLK_rank'] + league_stat['TO_rank']
league_stat['Roto_rank']=league_stat['Total'].rank(ascending=True, method='min') 
league_stat
#print(league_stat['Total'])

,Team,FG%,FT%,3PTM,PTS,REB,AST,ST,BLK,TO,...,FT%_rank,3PTM_rank,PTS_rank,REB_rank,AST_rank,ST_rank,BLK_rank,TO_rank,Total,Roto_rank
0,Sandro,0.461,0.645,53.0,476.0,220.0,133.0,40.0,22.0,64.0,...,1.0,3.0,3.0,7.0,2.0,7.0,5.0,5.0,37.0,3.0
1,$100 All in Jokic!!!,0.478,0.809,55.0,592.0,209.0,141.0,37.0,28.0,68.0,...,5.0,4.0,6.0,6.0,4.0,5.0,7.0,3.0,46.0,6.0
2,Happy Bird,0.418,0.846,61.0,464.0,141.0,139.0,48.0,15.0,59.0,...,6.0,5.0,2.0,2.0,3.0,8.0,2.0,6.0,35.0,2.0
3,Huyz Team,0.446,0.807,74.0,584.0,183.0,145.0,31.0,25.0,50.0,...,4.0,7.0,4.0,3.0,6.0,1.0,6.0,7.0,41.0,4.0
4,Luka Magic77,0.489,0.879,49.0,594.0,194.0,186.0,37.0,15.0,67.0,...,7.0,2.0,7.0,4.0,8.0,5.0,2.0,4.0,46.0,6.0
5,Tân Mai Pride,0.489,0.787,75.0,729.0,274.0,144.0,34.0,34.0,77.0,...,2.0,8.0,8.0,8.0,5.0,3.0,8.0,1.0,50.0,8.0
6,Cuckoo,0.470,0.802,69.0,586.0,197.0,173.0,36.0,20.0,70.0,...,3.0,6.0,5.0,5.0,7.0,4.0,4.0,2.0,41.0,4.0
7,phuctru0ng,0.420,0.915,32.0,317.0,128.0,107.0,33.0,7.0,47.0,...,8.0,1.0,1.0,1.0,1.0,2.0,1.0,8.0,25.0,1.0


#### Make the weekly stats table

In [35]:
def highlight_max(s):
    is_max=(s == s.nlargest(3)[0]) | (s == s.nlargest(3)[1]) | (s == s.nlargest(3)[2]) 
    return ['background-color: lightgreen' if v else '' for v in is_max]
   
def highlight_min(s):
    is_min =(s == s.nsmallest(3)[0]) | (s == s.nsmallest(3)[1]) | (s == s.nsmallest(3)[2])
    return ['background-color: pink' if v else '' for v in is_min]
def highlight_max_to(s):
    is_max=(s == s.nlargest(3)[0]) | (s == s.nlargest(3)[1]) | (s == s.nlargest(3)[2]) 
    return ['background-color: pink' if v else '' for v in is_max]
def highlight_min_to(s):
    is_min =(s == s.nsmallest(3)[0]) | (s == s.nsmallest(3)[1]) | (s == s.nsmallest(3)[2])
    return ['background-color: lightgreen' if v else '' for v in is_min]

head_to_head_stats = league_stat.iloc[:, 0:11]
head_to_head_stats.set_index('Team', inplace=True, drop=True)
head_to_head_stats.index.names = ['']



df_styled1 = head_to_head_stats.style.set_properties(**{})\
.set_precision(0).format({
    'FG%': '{:,.3f}'.format,
    'FT%': '{:,.3f}'.format,
    '3PTM': '{:,.0f}'.format,
    'PTS': '{:,.0f}'.format,
    'REB': '{:,.0f}'.format,
    'AST': '{:,.0f}'.format,
    'ST': '{:,.0f}'.format,
    'BLK': '{:,.0f}'.format,
    'TO': '{:,.0f}'.format,
    'Score': '{:,.0f}'.format,
}).apply(highlight_max, subset=['FG%','FT%','3PTM','PTS','REB','AST','ST','BLK']) \
.apply(highlight_min,subset=['FG%','FT%','3PTM','PTS','REB','AST','ST','BLK'])\
.apply(highlight_max_to, subset=['TO']).apply(highlight_min_to, subset=['TO'])\
.set_caption("\U0001f3c6"+name[0]+" Stats Week "+ str(week_plot)).set_table_styles([{
    'selector': 'caption',
    'props': [
        ('font-size', '20px'),['color', '#e6b800'],['font-weight', 'bold']
    ]
}])
##- C1: 6730 999999
##- SC: 35992 "\U0001f3c6"+ e6b800|
##- Dynasty: 4273 660033
##- PL: 36389 2f2fa7
##- SA: 35254 0066ff
##- CSL: 35264 ff8c66
##- Bundes: 35898 d966ff
##- VL: 43386 ff6666
##- L2: 43399 009933
#df_styled1
title = name[0]+"Stats"
dfi.export(df_styled1,str(title) +'.png')

/var/folders/nx/qyxw6n0n49x907bq5rmqhwsc0000gn/T/ipykernel_37065/2165315918.py:21: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  df_styled1 = head_to_head_stats.style.set_properties(**{})\
[2024-03-11 17:26:17,793 DEBUG] [PIL.PngImagePlugin.PngImagePlugin.call] STREAM b'IHDR' 16 13
[2024-03-11 17:26:17,795 DEBUG] [PIL.PngImagePlugin.PngImagePlugin.call] STREAM b'iCCP' 41 297
[2024-03-11 17:26:17,796 DEBUG] [PIL.PngImagePlugin.PngImagePlugin.chunk_iCCP] iCCP profile name b'Skia'
[2024-03-11 17:26:17,798 DEBUG] [PIL.PngImagePlugin.PngImagePlugin.chunk_iCCP] Compression method 0
[2024-03-11 17:26:17,799 DEBUG] [PIL.PngImagePlugin.PngImagePlugin.call] STREAM b'IDAT' 350 8192


#### Make the roto rank table

In [36]:
def highlight_max(s):
    is_max=(s == s.nlargest(3)[0]) | (s == s.nlargest(3)[1]) | (s == s.nlargest(3)[2]) 
    return ['background-color: lightgreen' if v else '' for v in is_max]
def highlight_min(s):
    is_min =(s == s.nsmallest(3)[0]) | (s == s.nsmallest(3)[1]) | (s == s.nsmallest(3)[2])
    return ['background-color: pink' if v else '' for v in is_min]

stat_name = ["Team","FG%","FT%","3PTM","PTS","REB","AST","ST","BLK","TO","Total","Rank"]



roto_stats = league_stat.iloc[:,0:1].join(league_stat.iloc[:,11:22])
roto_stats.columns = stat_name
roto_stats.set_index('Team', inplace=True, drop=True)
roto_stats.index.names = ['']
roto_stats.sort_values(by=['Total'], inplace=True, ascending=False)

df_styled2 = roto_stats.style.set_precision(0).apply(highlight_max) \
.apply(highlight_min).set_caption("\U0001f3c6"+name[0]+" Roto Rank Week "+ str(week_plot)).set_table_styles([{
    'selector': 'caption',
    'props': [
        ('font-size', '20px'),['color', '#e6b800'],['font-weight', 'bold']
    ]
}])

##- C1: 6730 999999
##- SC: 35992 U0001f3c6 e6b800
##- Dynasty: 4273 660033
##- PL: 36389 2f2fa7
##- SA: 35254 0066ff
##- CSL: 35264 ff8c66
##- Bundes: 35898 d966ff
##- VL: 43386 ff6666
##- L2: 43399 009933
title = name[0]+"RotoRank"
dfi.export(df_styled2,str(title) +'.png')

/var/folders/nx/qyxw6n0n49x907bq5rmqhwsc0000gn/T/ipykernel_37065/1859652925.py:18: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  df_styled2 = roto_stats.style.set_precision(0).apply(highlight_max) \
[2024-03-11 17:26:30,691 DEBUG] [PIL.PngImagePlugin.PngImagePlugin.call] STREAM b'IHDR' 16 13
[2024-03-11 17:26:30,693 DEBUG] [PIL.PngImagePlugin.PngImagePlugin.call] STREAM b'iCCP' 41 297
[2024-03-11 17:26:30,695 DEBUG] [PIL.PngImagePlugin.PngImagePlugin.chunk_iCCP] iCCP profile name b'Skia'
[2024-03-11 17:26:30,696 DEBUG] [PIL.PngImagePlugin.PngImagePlugin.chunk_iCCP] Compression method 0
[2024-03-11 17:26:30,698 DEBUG] [PIL.PngImagePlugin.PngImagePlugin.call] STREAM b'IDAT' 350 8192
